In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install transformers==4.52.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 88.4 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 28.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.0a1 which is incompatible.


In [3]:
!pip install youtube-transcript-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.1/485.1 kB 9.6 MB/s eta 0:00:00ta 0:00:01


In [4]:
from huggingface_hub import login
login()

In [5]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

2025-10-20 12:02:45.621343: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760961765.883602      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760961765.954341      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [6]:
from urllib.parse import urlparse, parse_qs
from youtube_transcript_api import YouTubeTranscriptApi

def extract_video_id(url: str) -> str:
    """
    Extract the YouTube video ID from a URL.
    Raises ValueError if no 'v' parameter is found.
    """
    parsed = urlparse(url)
    qs = parse_qs(parsed.query)
    video_ids = qs.get('v')
    
    if not video_ids:
        raise ValueError(f"No video id found in URL: {url}")
    return video_ids[0]

In [7]:
def chunking_text(text, max_length = 700):
    """
    Splitting text into smaller parts
    """
    return [text[i:i+max_length] for i in range(0, len(text), max_length)]

In [8]:
LINK = "https://www.youtube.com/watch?v=Fa_V9fP2tpU"

In [10]:
if __name__ == "__main__":
    url = LINK
    
    video_id = extract_video_id(url)
    
    api = YouTubeTranscriptApi()
    fetched = api.fetch(video_id, languages=['en']) # returns a FetchedTranscript
    
    text = "\n".join(snippet.text for snippet in fetched)

    chunks = chunking_text(text)

    summariesed_chunks = [summarizer(chunk, max_length=50, min_length=20, do_sample=False)[0]['summary_text'] for chunk in chunks[0:10]]
    output = summarizer(" ".join(summariesed_chunks), max_length=200, min_length=50, do_sample=False)[0]['summary_text']

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [12]:
print(output)

Artificial intelligence is the capability of machines to perform tasks that typically require human intelligence. AI aims to mimic human cognitive functions through various techniques including machine learning. Machine learning algorithms can identify patterns and relationships within data making predictions or decisions based on new unseen information. Here a list of all basic machine learning terms in 22 minutes.
